# IBM Watson

Paso 0: Configurar Watson (instalar librerías, armar usuario y conseguir los permisos necesarios).

In [1]:
!pip install --upgrade "ibm-watson>=4.4.0"

Requirement already up-to-date: ibm-watson>=4.4.0 in c:\users\nereida.combariza\.conda\envs\watson2\lib\site-packages (4.4.1)


In [2]:
!pip install --upgrade watson-developer-cloud
!pip install --upgrade watson-machine-learning-client

Requirement already up-to-date: watson-developer-cloud in c:\users\nereida.combariza\.conda\envs\watson2\lib\site-packages (2.10.1)
Requirement already up-to-date: watson-machine-learning-client in c:\users\nereida.combariza\.conda\envs\watson2\lib\site-packages (1.0.378)


### Configurando la API de watson
En este paso van a precisar usar el api_key que se generan en la página de IBM 

Revisar  el vídeo https://www.acamica.com/clases/10669//machine-learning-resource

## Creando un nuevo recurso

Ahora deberás crear el recurso Machine Learning

    * Cliquear donde dice Create resource. Esto desplegará una lista de todos los recursos disponibles.
    * Seleccionar Machine Learning. Este servicio nos permitirá realizar el deploy de nuestro modelo en la nube.
    
Obtener credenciales

    * Dirigirse a la sección Service credentials. El botón podrás encontrarlo al costado superior izquierdo.
    * Una vez allí podrás crear nuevas credenciales para tu servicio cliqueando en new credentials.
    * Las credenciales que aparezcan serán las que utilizarás para completar en tu notebook de la siguiente clase. 
    Guardalas.

Ahora podrás subir tu modelo a la nube y utilizar las funcionalidades de la API Machine Learning.

In [3]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd

# Eso aparecía a en el notebook que se bajan de la página (ya no funciona):
# from watson_developer_cloud import NaturalLanguageUnderstandingV1
# from watson_developer_cloud.natural_language_understanding_v1 import Features, EmotionOptions, SentimentOptions

# Esta es la nueva forma de hacerlo
from ibm_watson import NaturalLanguageUnderstandingV1
from ibm_watson.natural_language_understanding_v1 import Features, CategoriesOptions, EmotionOptions, SentimentOptions

In [4]:
import json
from ibm_cloud_sdk_core.authenticators import IAMAuthenticator

In [5]:
authenticator = IAMAuthenticator('Ingrese_Su_Key')

natural_language_understanding = NaturalLanguageUnderstandingV1(
    version='2019-07-12',
    authenticator=authenticator,
)

natural_language_understanding.set_service_url('https://gateway-syd.watsonplatform.net/natural-language-understanding/api')

### Probando a Watson

Exploremos el tipo de objeto que nos devuelve Watson y como funciona para distintas frases

In [6]:
# Texto objetivo
text = 'I want to see the dawn tomorrow because today I could not.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='en')
resultado = response.result
resultado

{'usage': {'text_units': 1, 'text_characters': 58, 'features': 2},
 'sentiment': {'document': {'score': -0.73691, 'label': 'negative'}},
 'language': 'en',
 'emotion': {'document': {'emotion': {'sadness': 0.24932,
    'joy': 0.318841,
    'fear': 0.322898,
    'disgust': 0.044808,
    'anger': 0.118712}}}}

Para tomar sólo alguno de los resultados, podemos acceder al diccionario

In [7]:
type(resultado)

dict

In [8]:
print(resultado['usage'])

{'text_units': 1, 'text_characters': 58, 'features': 2}


In [9]:
print(resultado['sentiment'])

{'document': {'score': -0.73691, 'label': 'negative'}}


In [10]:
print(resultado['language'])

en


In [11]:
print(resultado['sentiment']['document']['score'])

-0.73691


Probamos ahora un texto en español

In [12]:
# Texto objetivo
text = 'Este texto no tiene ningún objetivo, vive en la soledad que genera saber que su existencia no tiene un proposito determinado.'

# Importamos el modelo que vamos a usar y le avisamos que features queremo que devuelva
# y en que lenguaje está el texto (esto es mejor que dejarlo adivinar)
response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='es')
resultado = response.result
resultado

{'usage': {'text_units': 1, 'text_characters': 125, 'features': 1},
 'sentiment': {'document': {'score': 0, 'label': 'neutral'}},
 'language': 'es',
 'warnings': ['emotion: unsupported text language: es']}

### Dataset de películas
Este es el dataset propuesto por la plataforma de Acámica, al estár en ingles permite explotar todas las funcionalidades de Watson. Carguen distintos reviews y comprueben si la devolución del analisis de emosiones de Watson coinciden con lo que perciben del texto.

In [13]:
import sklearn
from sklearn.datasets import load_files
moviedir = r'.\movie_reviews' 
movie_reviews = load_files(moviedir, shuffle=True)

In [14]:
def analisis_watson(text):
    text = text.decode("utf-8")
    response = natural_language_understanding.analyze(text=text,features=Features(emotion=EmotionOptions(), sentiment=SentimentOptions()),language='es')
    return response

In [15]:
print(analisis_watson(movie_reviews.data[7]))

{
    "result": {
        "usage": {
            "text_units": 1,
            "text_characters": 5584,
            "features": 1
        },
        "sentiment": {
            "document": {
                "score": 0.588432,
                "label": "positive"
            }
        },
        "language": "es",
        "warnings": [
            "emotion: unsupported text language: es"
        ]
    },
    "headers": {
        "_store": {
            "content-type": [
                "Content-Type",
                "application/json; charset=utf-8"
            ],
            "content-length": [
                "Content-Length",
                "271"
            ],
            "x-powered-by": [
                "X-Powered-By",
                "Express"
            ],
            "cache-control": [
                "Cache-Control",
                "no-cache, no-store"
            ],
            "x-dp-watson-tran-id": [
                "x-dp-watson-tran-id",
                "eb40ff5946b3b3b5a6

In [16]:
print(movie_reviews.data[4])

b"kolya is one of the richest films i've seen in some time . \nzdenek sverak plays a confirmed old bachelor ( who's likely to remain so ) , who finds his life as a czech cellist increasingly impacted by the five-year old boy that he's taking care of . \nthough it ends rather abruptly-- and i'm whining , 'cause i wanted to spend more time with these characters-- the acting , writing , and production values are as high as , if not higher than , comparable american dramas . \nthis father-and-son delight-- sverak also wrote the script , while his son , jan , directed-- won a golden globe for best foreign language film and , a couple days after i saw it , walked away an oscar . \nin czech and russian , with english subtitles . \n"


### Dataset de Noticias Infobae de hoy

Este dataset se lo preparamos para que puedan explorar cómo funciona Watson en textos en español. Algunas de las funcionalidades se pierden (no reporta las emociones), pero sí clasifica como positivo o negativo los cuerpos de texto. Noten que otra estrategia podría ser traducir el texto y luego aplicarle el algoritmo de Watson en inglés. Esta otra estrategia no siempre funciona bien, depende mucho de la calidad de la traducción.

In [17]:
!pip install -U pyarrow

Requirement already up-to-date: pyarrow in c:\users\nereida.combariza\.conda\envs\watson2\lib\site-packages (0.17.0)


In [18]:
nombre_archivo = '2019_08_21.feather'

Exploremos el dataset

In [19]:
# Quiero ver el sentimiento de los cuerpos de la noticia.
noticias_hoy = pd.read_feather(nombre_archivo)
noticias_hoy

,Fecha,Id-Fecha,Subtitulo,Tema,Texto,Titulo
0,2019/08/21,0,"El tipo de cambio cerró con leve baja, pero lo...",economia,Hernán Lacunza logró sortear una prueba compl...,Hernán Lacunza consiguió una tregua con el dól...
1,2019/08/21,1,El encuentro será a las 11. Cuál puede ser el ...,politica,Hoy a las 11 de la mañana se verán las caras ...,Los diputados de Cambiemos se reunirán por pri...
2,2019/08/21,2,"En 1974, el ex juez Jorge Vicente Quiroga fue ...",sociedad,El domingo 28 de abril de 1974 Jorge Vicente ...,El asesinato del juez Quiroga: sabía que estab...
3,2019/08/21,3,"""Llaman todos"", aseguran los allegados al cand...",economia,"""Acá llaman todos, todo el mundo quiere venir...","Los empresarios en su laberinto: dudas, enojos..."
4,2019/08/21,4,"Hacienda mostró una vez más el rol de ""otros i...",economia,"Unas de las preocupaciones de ""los mercados"" ...",Las ganancias del Banco Central permitirán cum...
5,2019/08/21,5,Osvaldo Agosto era integrante de la Juventud P...,sociedad,"Sus ojos claros, vivaces, y esa sonrisa entre...","""Yo robé el sable corvo de San Martín"": el rel..."
6,2019/08/21,6,Gustavo Figueroa fue separado del cargo y some...,sociedad,"Gustavo Figueroa, quien prestaba servicios de...",Tucumán: trabajó durante nueve años como médic...
7,2019/08/21,7,El artista mexicano participó en Buenos Aires ...,sociedad,Eduardo Verástegui ya filmaba en Hollywood y ...,"La asombrosa conversión de Eduardo Verástegui,..."
8,2019/08/21,8,En las imágenes se ve a Jorge Martín Gómez par...,sociedad,Uno de los principales temas que dominó la co...,El video que muestra la amenaza del hombre que...
9,2019/08/21,9,La gobernadora bonaerense dejará de recibir 4....,politica,María Eugenia Vidal decidió postergar todos l...,María Eugenia Vidal postergó los anuncios econ...


In [20]:
cuerpos = noticias_hoy.Texto
titulos = noticias_hoy.Titulo
print(len(cuerpos))

24


Vamos a calcular el sentimiento para cada una de las noticias y agregarlo al dataframe 

In [21]:
score_cuerpos = []

for este_texto in cuerpos:
    
    # Quitamos del texto este símbolo que indica final de parrafo
    este_texto = este_texto.replace('\n','')
    
    # Analisis sentimiento del cuerpo
    # Le pedimos solo el sentimiento, y le avisamos que está en español
    response = natural_language_understanding.analyze(text=este_texto,features=Features(sentiment=SentimentOptions()),language='es')
    resultado = response.result
    score_cuerpos.append(resultado['sentiment']['document']['score'])

In [22]:
print(score_cuerpos)

[-0.628192, 0.703094, -0.603858, -0.361676, 0.357864, -0.413688, -0.851592, 0.304766, -0.740305, 0, -0.607553, 0.276372, 0, -0.621652, -0.490642, -0.40268, -0.85545, 0, 0.583955, -0.642936, 0.348123, -0.517949, -0.416298, -0.488772]


In [23]:
# Finalmente agregamos la columna al dataset
noticias_hoy['Sentimiento'] = score_cuerpos
noticias_hoy

,Fecha,Id-Fecha,Subtitulo,Tema,Texto,Titulo,Sentimiento
0,2019/08/21,0,"El tipo de cambio cerró con leve baja, pero lo...",economia,Hernán Lacunza logró sortear una prueba compl...,Hernán Lacunza consiguió una tregua con el dól...,-0.628192
1,2019/08/21,1,El encuentro será a las 11. Cuál puede ser el ...,politica,Hoy a las 11 de la mañana se verán las caras ...,Los diputados de Cambiemos se reunirán por pri...,0.703094
2,2019/08/21,2,"En 1974, el ex juez Jorge Vicente Quiroga fue ...",sociedad,El domingo 28 de abril de 1974 Jorge Vicente ...,El asesinato del juez Quiroga: sabía que estab...,-0.603858
3,2019/08/21,3,"""Llaman todos"", aseguran los allegados al cand...",economia,"""Acá llaman todos, todo el mundo quiere venir...","Los empresarios en su laberinto: dudas, enojos...",-0.361676
4,2019/08/21,4,"Hacienda mostró una vez más el rol de ""otros i...",economia,"Unas de las preocupaciones de ""los mercados"" ...",Las ganancias del Banco Central permitirán cum...,0.357864
5,2019/08/21,5,Osvaldo Agosto era integrante de la Juventud P...,sociedad,"Sus ojos claros, vivaces, y esa sonrisa entre...","""Yo robé el sable corvo de San Martín"": el rel...",-0.413688
6,2019/08/21,6,Gustavo Figueroa fue separado del cargo y some...,sociedad,"Gustavo Figueroa, quien prestaba servicios de...",Tucumán: trabajó durante nueve años como médic...,-0.851592
7,2019/08/21,7,El artista mexicano participó en Buenos Aires ...,sociedad,Eduardo Verástegui ya filmaba en Hollywood y ...,"La asombrosa conversión de Eduardo Verástegui,...",0.304766
8,2019/08/21,8,En las imágenes se ve a Jorge Martín Gómez par...,sociedad,Uno de los principales temas que dominó la co...,El video que muestra la amenaza del hombre que...,-0.740305
9,2019/08/21,9,La gobernadora bonaerense dejará de recibir 4....,politica,María Eugenia Vidal decidió postergar todos l...,María Eugenia Vidal postergó los anuncios econ...,0.000000


Exploremos un poco los resultados. Veamos el promedio de sentimiento de todas las noticias (esto nos va a dar una idea del ánimo general del día). Luego busquemos la noticia más positiva y la más negativa, para ver si coincidimos con el criterio.

In [24]:
todos_los_valores = noticias_hoy['Sentimiento']
animo_general = np.mean(todos_los_valores)
print(animo_general)

-0.252877875


In [25]:
noticia_positiva = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmin()]
print(noticia_positiva.Titulo)
print(noticia_positiva.Sentimiento)
#print(noticia_positiva.Subtitulo)
print(noticia_positiva.Texto)

Condenaron a 12 años de prisión a dos hombres por violar a una joven a la salida de un boliche
-0.85545
 Dos hombres fueron condenados este martes durante un juicio oral a la pena de 12 años de prisión por haber violado a una joven en 2013, en un el partido bonaerense de Quilmes, tras haber concurrido a un boliche. 
  La decisión por unanimidad del Tribunal Oral en lo Criminal 1 del Departamento Judicial de Quilmes recayó sobre Hugo Armando Barroca de 37 años y Sergio Miguel Bigón de 35, quienes fueron acusados del delito de "abuso sexual con acceso carnal". 
  A su vez, los jueces ordenaron por mayoría la detención inmediata de ambos, quienes habían llegado al debate oral con prisión domiciliaria. 
  En su momento la fiscal Mariel Calviño había solicitado en su alegato que sean condenados a 15 años de prisión, mientras que los defensores de los imputados solicitaron la absolución porque consideraron que no hubo suficientes pruebas para que sean hallados culpables. 
  La fiscal basó su

In [26]:
noticia_negativa = noticias_hoy.loc[noticias_hoy['Sentimiento'].idxmin()]
print(noticia_negativa.Titulo)
print(noticia_negativa.Sentimiento)
#print(noticia_negativa.Subtitulo)
#print(noticia_negativa.Texto)

Condenaron a 12 años de prisión a dos hombres por violar a una joven a la salida de un boliche
-0.85545
